In [29]:
from sklearn import datasets
# coding: utf-8
import datetime
import gc
import math
import os
import seaborn as sns
import time
import warnings
from contextlib import contextmanager

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import (auc, average_precision_score, confusion_matrix,
                             mean_squared_error, precision_recall_curve,
                             roc_curve)
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error

In [39]:
iris = datasets.load_iris(as_frame=True)
target_names = iris['target_names']
df = iris['frame']

In [54]:
params ={
    'task': 'train',
    'boosting': 'goss',
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.008,
    'subsample': 0.9855232997390695,
    'max_depth': 7,
    'top_rate': 0.9064148448434349,
    'num_leaves': 63,
    'min_child_weight': 41.9612869171337,
    'other_rate': 0.0721768246018207,
    'reg_alpha': 9.677537745007898,
    'colsample_bytree': 0.5665320670155495,
    'min_split_gain': 9.820197773625843,
    'reg_lambda': 8.2532317400459,
    'min_data_in_leaf': 21,
    'verbose': -1,
    'seed':0,
    'bagging_seed':0,
    'drop_seed':0,
    }


In [55]:
folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=42)

In [56]:
train_df = df[df.columns[0:4]]
target = df['target']

In [72]:
# LightGBM GBDT with KFold or Stratified KFold
def oof_lgb(train_df, target, folds, params):
    print(f"Starting LightGBM {train_df.shape}")
    # Cross validation model
    cv_score = 0
    # # Create arrays and dataframes to store results
    # cv_score = 0
    oof_preds = np.zeros(train_df.shape[0])
    # sub_preds = np.zeros(test_df.shape[0])
    # feature_importance_df = pd.DataFrame()
    # feats = [f for f in train_df.columns if f not in FEATS_EXCLUDED]

    # k-fold

    for _fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, target)):
        train_x, train_y = train_df.iloc[train_idx], target.iloc[train_idx]
        valid_x, valid_y = train_df.iloc[valid_idx], target.iloc[valid_idx]
        
            # set data structure
        lgb_train = lgb.Dataset(train_x,
                                label=train_y,
                                free_raw_data=False)
        lgb_test = lgb.Dataset(valid_x,
                               label=valid_y,
                               free_raw_data=False)
        

        clf = lgb.train(
                        params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_test],
                        valid_names=['train', 'test'],
                        num_boost_round=10000,
                        )
        oof_preds[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration)
        cv_score += mean_squared_error(valid_y, oof_preds[valid_idx]) /folds.n_splits
    return cv_score / folds.n_splits

In [73]:
oof_lgb(train_df, target, folds, params)

Starting LightGBM (150, 4)


0.0713961225385033

In [15]:
#     # reg.save_model(output_file_path + str(n_fold)+ 'all_train_model.txt', num_iteration = reg.best_iteration)
#     sub_preds += reg.predict(test_df[feats], num_iteration=reg.best_iteration) / folds.n_splits

#     #feature importance
#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["feature"] = feats
#     fold_importance_df["importance"] = np.log1p(reg.feature_importance(importance_type='gain', iteration=reg.best_iteration))
#     fold_importance_df["fold"] = n_fold + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
#     print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
#     cv_score += rmse(valid_y, oof_preds[valid_idx]) /folds.n_splits
#     del reg, train_x, train_y, valid_x, valid_y
#     gc.collect()

# # display importances
# print(cv_score)
# # display_importances(feature_importance_df)

# if not debug:
#     # save submission file
#     test_df.loc[:,'target'] = sub_preds
#     test_df = test_df.reset_index()
#     test_df[['card_id', 'target']].to_csv(output_file_path + 'all_train_model_submission.csv', index=False)